In [1]:
import numpy as np
from ipalm import *
import matplotlib.pyplot as plt

In [2]:
##### solve sparse PCA as test

### generate toy matrix to factorize

M=33 #dimensions
N=25 #dimensions
r=6 #rank of matrix


#subscript of 't' denotes true parameter used to generate toy data

p_sparse=0.05

#let's make these things sparse
sp_mask_t=np.random.binomial(1,p_sparse,size=[M, r])

U_t=np.random.normal(size=[M, r])
V_t=np.random.normal(size=[r, N])



U_t=U_t/np.linalg.norm(U_t,axis=0)
V_t=V_t/np.expand_dims(np.linalg.norm(V_t,axis=1),axis=1)

s_t=np.random.uniform(size=[r,])
s_t=sorted(s_t)

S_t=np.zeros(shape=[r, r])
for i in range(r):
            S_t[i,i]=s_t[i]


P_t=np.dot(np.dot(U_t,S_t),V_t)


plt.figure()
plt.imshow(P_t, interpolation='none')


plt.show()


In [6]:
### the iPALM algorithm
### no backtracking

### this specific problem
# solve 1/2*|| Y-Xa*Xb ||_F^2 + lam*||xa||_1 + lam/2*||xb||_2^2


### inputs

#inits
xa0=np.random.normal(size=[M*r,])
xb0=np.random.normal(size=[r*N,])

#functions
grad_a=lambda xs, opts: \
np.reshape(np.dot(opts['Y']-np.dot(np.reshape(xs[0], newshape=[opts['M'], opts['r']]),\
                                   np.reshape(xs[1], newshape=[opts['r'], opts['N']])),\
                  -np.reshape(xs[1], newshape=[opts['r'], opts['N']]).T),\
           [opts['M']*opts['r'],])

grad_b=lambda xs, opts: \
np.reshape(np.dot(-np.reshape(xs[0], newshape=[opts['M'], opts['r']]).T,\
                  opts['Y']-np.dot(np.reshape(xs[0], newshape=[opts['M'], opts['r']]),\
                                   np.reshape(xs[1], newshape=[opts['r'], opts['N']]))),\
           [opts['r']*opts['N'],])

prox_a=lambda v, opts: \
np.sign(v)*np.maximum(0.0, np.abs(v)-opts['lam'])

prox_b=lambda v, opts: \
v/(1.0+opts['lam'])

#optimization params
tau=0.01 #step size
nit=1e3 #max iterations before termination
tol=1e-4 #error threshold for termination

opts={}
opts['lam']=0.0001;
opts['r']=r
opts['M']=M
opts['N']=N
opts['Y']=P_t

opts['tau']=tau
opts['nit']=nit
opts['tol']=tol

grads=[grad_a, grad_b]
proxs=[prox_a, prox_b]

lip_a = lambda xs, opts: np.linalg.norm(xs[1])**2
lip_b = lambda xs, opts: np.linalg.norm(xs[0])**2

lips=[lip_a, lip_b]
opts['lips']=lips
xs2 = ipalm(grads, proxs, [M*r, r*N], opts)
[xa2, xb2] = xs2


P_h=np.dot(np.reshape(xa2,[M,r]),np.reshape(xb2,[r,N]))

In [9]:
cmi=np.min([P_t,P_h])
cma=np.max([P_t,P_h])

plt.figure()
plt.imshow(P_h,interpolation='none')
plt.clim(cmi,cma)
plt.colorbar()

plt.figure()
plt.imshow(P_t,interpolation='none')
plt.clim(cmi,cma)
plt.colorbar()

plt.figure()
plt.imshow(P_t-P_h,interpolation='none')
plt.clim(cmi,cma)
plt.colorbar()

plt.show()
